In [67]:
suppressPackageStartupMessages(suppressWarnings({
    library(rhdf5)
    library(Matrix)
    library(CytoTRACE)
    library(ggplot2)
    library(viridis)
}))

In [5]:
prj.dir <- "/scratch/gpfs/KANG/sereno/spatialstem"
src.dir <- file.path(prj.dir, "sourcefiles")
raw.dir <- file.path(src.dir, "raw")
had.dir <- file.path(src.dir, "h5ad")
int.dir <- file.path(prj.dir, "intermediates")
fig.dir <- file.path(prj.dir, "figs")
csv.dir <- file.path(src.dir, "csv")
exp.dir <- file.path(csv.dir, "exp")
cor.dir <- file.path(csv.dir, "coords")

In [72]:
# Function for opening an h5ad file as a formatted 10x object.
exp.files <- list.files(exp.dir)
exp.paths <- paste0(exp.dir, "/", exp.files)
cor.files <- list.files(cor.dir)
cor.paths <- paste0(cor.dir, "/", cor.files)

In [73]:
exp.path <- exp.paths[1]
cor.path <- cor.paths[1]
print(exp.path)
print(cor.path)

[1] "/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/csv/exp/p01_fetalliver_slice1_exp.csv"
[1] "/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/csv/coords/p01_fetalliver_slice1_coords.csv"


In [74]:
# Load in expression data
expr <- data.frame(read.table(exp.path, header=TRUE, sep=","))
gene.names <- expr[, 1]

In [75]:
# Remove duplicate gene rows and reformat matrix.
duped.genes <- duplicated(gene.names)
expr.filt <- expr[!duped.genes, ]
rownames(expr.filt) <- expr.filt[, 1]
expr.filt[, 1] <- NULL
colnames(expr.filt) <- gsub("X", "", colnames(expr.filt))

In [ ]:
n.cells <- ncol(expr.filt)
ct.results <- CytoTRACE(expr.filt, ncores=16, enableFast=FALSE)
ct.vals <- ct.results$CytoTRACE

The number of cells in your dataset is less than 3,000. Fast mode has been disabled.

Warning message in CytoTRACE(expr.filt, ncores = 16, enableFast = FALSE):
“929 genes have zero expression in the matrix and were filtered”
CytoTRACE will be run on 1 sub-sample(s) of approximately 2835 cells each using 1 / 16 core(s)

Pre-processing data and generating similarity matrix...



In [ ]:
coords <- read.table(cor.path, header=FALSE, sep=",")
ct.df <- data.frame(x=coords[, 1], y=coords[, 2], cytotrace=ct.vals)

In [ ]:
ggplot(data=ct.df, aes(x=x, y=y, color=cytotrace)) + 
    geom_point(size=1) + 
    theme_classic() + 
    scale_color_viridis(option="plasma")